In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/online-retail-ii-uci/online_retail_II.csv")

In [ ]:
df.head()
df.info()

In [ ]:
df.InvoiceDate.value_counts().head()

In [ ]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)

In [ ]:
df.InvoiceDate.dtypes

In [ ]:
df.info()

In [ ]:
import datetime as dt

In [ ]:
#sadece 201-2011 verisini alıyorduk
df = df[(df["InvoiceDate"].dt.year.values == 2010) | (df["InvoiceDate"].dt.year.values == 2011)]

In [ ]:
df.InvoiceDate.dt.year.value_counts()

## Veri Önişleme

In [ ]:
!pip install mlxtend

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [ ]:
def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe


In [ ]:
df = retail_data_prep(df)

## ARL Veri Yapısını hazırlama

In [ ]:
df_ger = df[df.Country == "Germany"]

In [ ]:
df_ger.Country.value_counts()

In [ ]:
df_ger.head()

In [ ]:
# arl yapısına çeviriyoruz. satırda invoice ,sütunlarda eşsiz ürün tanımları ve değerler satın alınma frekansları olacak şekilde
df_ger.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().fillna(0).head(20)

In [ ]:
# 0'dan büyük, 1 harici değerleri 1'e eşitliyoruz
df_ger.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0).head()

In [ ]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return(dataframe.groupby(["Invoice", "StockCode"])["Quantity"].sum().unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0))
    else:
        return(dataframe.groupby(["Invoice", "Description"])["Quantity"].sum().unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0))

In [ ]:
ger_inv_pr_df = create_invoice_product_df(df_ger, id=True)

In [ ]:
ger_inv_pr_df.head()

In [ ]:
def check_id(dataframe, StockCode):
    return(dataframe[dataframe["StockCode"] == StockCode]["Description"].values[1])

## görev3

In [ ]:
check_id(df_ger, "21987")

In [ ]:
check_id(df_ger, "23235")

In [ ]:
check_id(df_ger, "22747")

In [ ]:
ger_inv_pr_df### post değerlerinin nereden çıktığını görmek için bir fonksiyon yazmak

## Birliktelik Kuralının Çıkarılması

In [ ]:
frequent_itemsets = apriori(ger_inv_pr_df, min_support=0.01, use_colnames=True)


In [ ]:
frequent_itemsets.sort_values("support", ascending=False).head(15)

In [ ]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)


In [ ]:
rules.sort_values("support", ascending=False).head(20)

### 4/5 Kullanıcılara ürün önerisi/önerilen ürünlerin isimleri

In [ ]:
product_id = "21987"
check_id(df, product_id)

In [ ]:
sorted_rules = rules.sort_values("confidence", ascending=False)

In [ ]:
recommendation_list = []

for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

In [ ]:
recommendation_list[0:10]

In [ ]:
check_id(df_ger,"21086")

In [ ]:
product_id = "23235"
check_id(df, product_id)

In [ ]:
sorted_rules = rules.sort_values("confidence", ascending=False)

In [ ]:
recommendation_list = []

for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])
            


In [ ]:
recommendation_list[0:3]

In [ ]:
product_id = "22747"
check_id(df, product_id)

In [ ]:
recommendation_list = []

for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])
            

In [ ]:
recommendation_list[0:3]

In [ ]:
product_id = "22748"
check_id(df, product_id)

In [ ]:
product_id = "22745"
check_id(df, product_id)